In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from rapidfuzz import fuzz, process
import numpy as np
from collections import defaultdict

# Preprocessing functions
def preprocess_text(text):
    """Clean and standardize text."""
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def fuzzy_standardize(names, threshold=85):
    """Use fuzzy matching to group similar names efficiently."""
    standardized = {}
    processed = set()

    for name in names:
        if name in processed:
            continue
        
        best_match = process.extractOne(name, standardized.keys(), scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
        
        if best_match:
            standardized[name] = standardized[best_match[0]]
        else:
            standardized[name] = name  # Assign itself if no good match
        
        processed.add(name)
    
    return standardized

def create_key(name):
    """Create standardized key for grouping similar names."""
    name = preprocess_text(name)
    name = re.sub(r'\b(ltd|limited|co|company|ea|east\s+africa)\b', '', name)
    name = re.sub(r'[^a-z0-9]', '', name)
    return name

# Define known incorrect mappings
INCORRECT_MAPPINGS = {
    "Mapato Feeds": ["Mazao Feeds"],
    "murimi feeds": ["Mumbi feeds"],
}

def should_exclude_mapping(name1, name2):
    """Check if two names should not be mapped together."""
    for standard_name, incorrect_vars in INCORRECT_MAPPINGS.items():
        lower_standard = standard_name.lower()
        lower_incorrects = [x.lower() for x in incorrect_vars]
        if (name1.lower() == lower_standard and name2.lower() in lower_incorrects) or \
           (name2.lower() == lower_standard and name1.lower() in lower_incorrects):
            return True
    return False

# Clustering functions
def find_similar_names(name, names, threshold=85):
    """Find similar names using RapidFuzz."""
    return [other_name for other_name in names if fuzz.token_sort_ratio(name, other_name) >= threshold]

def cluster_names(names, similarity_threshold=85):
    """Cluster similar names using fuzzy matching."""
    clusters = defaultdict(list)
    remaining_names = set(names)
    
    while remaining_names:
        current_name = remaining_names.pop()
        current_cluster = [current_name]
        
        similar_names = find_similar_names(current_name, list(remaining_names), threshold=similarity_threshold)
        similar_names = [name for name in similar_names if not should_exclude_mapping(current_name, name)]
        
        for similar_name in similar_names:
            if similar_name in remaining_names:
                current_cluster.append(similar_name)
                remaining_names.remove(similar_name)
        
        standardized_name = current_cluster[0]
        clusters[standardized_name].extend(current_cluster)
    
    return clusters

def create_mapping_dict(clusters):
    """Create mapping dictionary from variations to standardized names."""
    return {var: std for std, vars in clusters.items() for var in vars}

# Data Cleaning & Standardization
def clean_and_standardize_data(input_file='products_2.csv', similarity_threshold=85):
    """Main function to clean and standardize the dataset."""
    print("Reading dataset...")
    df = pd.read_csv(input_file)
    
    # Drop rows with missing critical values
    df.dropna(subset=['product_manufacturer_name', 'product_name'], inplace=True)
    
    # Cluster manufacturer names
    print("Clustering manufacturer names...")
    manufacturer_names = df['product_manufacturer_name'].unique()
    manufacturer_clusters = cluster_names(manufacturer_names, similarity_threshold)
    manufacturer_mapping = create_mapping_dict(manufacturer_clusters)
    
    # Cluster product names
    print("Clustering product names...")
    product_names = df['product_name'].unique()
    product_clusters = cluster_names(product_names, similarity_threshold)
    product_mapping = create_mapping_dict(product_clusters)
    
    # Save mapping details
    print("Saving mappings to file...")
    with open('name_mappings_fuzzy.txt', 'w') as f:
        f.write("=== Manufacturer Name Mappings ===\n\n")
        for standard_name, variations in manufacturer_clusters.items():
            f.write(f"Standard Name: {standard_name}\n")
            f.write("Variations:\n")
            for var in sorted(variations):
                if var != standard_name:
                    f.write(f"  - {var}\n")
            f.write("\n")
        
        f.write("\n=== Product Name Mappings ===\n\n")
        for standard_name, variations in product_clusters.items():
            f.write(f"Standard Name: {standard_name}\n")
            f.write("Variations:\n")
            for var in sorted(variations):
                if var != standard_name:
                    f.write(f"  - {var}\n")
            f.write("\n")
    
    # Create standardized dataset
    print("Creating standardized dataset...")
    df['product_manufacturer_name_std'] = df['product_manufacturer_name'].map(manufacturer_mapping)
    df['product_name_std'] = df['product_name'].map(product_mapping)
    
    # Save standardized dataset
    output_file = 'products_standardized_fuzzy.csv'
    df.to_csv(output_file, index=False)
    
    print(f"\nOriginal unique manufacturer names: {len(manufacturer_names)}")
    print(f"Standardized unique manufacturer names: {len(df['product_manufacturer_name_std'].unique())}")
    print(f"Original unique product names: {len(product_names)}")
    print(f"Standardized unique product names: {len(df['product_name_std'].unique())}")
    
    return df

# Clustering with DBSCAN
def cluster_data(file_path='products_standardized_fuzzy.csv'):
    print("Loading standardized dataset...")
    df = pd.read_csv(file_path)
    df.dropna(subset=['product_name_std', 'product_manufacturer_name_std', 'product_category_name', 'product_type_numeric'], inplace=True)
    
    print("Encoding text features with TF-IDF...")
    vectorizer_product = TfidfVectorizer()
    vectorizer_manufacturer = TfidfVectorizer()
    vectorizer_category = TfidfVectorizer()

    product_tfidf = vectorizer_product.fit_transform(df['product_name_std'])
    manufacturer_tfidf = vectorizer_manufacturer.fit_transform(df['product_manufacturer_name_std'])
    category_tfidf = vectorizer_category.fit_transform(df['product_category_name'])
    
    print("Scaling numerical features...")
    scaler = StandardScaler()
    type_numeric_scaled = scaler.fit_transform(df[['product_type_numeric']])
    
    print("Combining features for clustering...")
    feature_matrix = np.hstack((product_tfidf.toarray(), manufacturer_tfidf.toarray(), category_tfidf.toarray(), type_numeric_scaled))
    
    print("Applying DBSCAN...")
    clustering = DBSCAN(eps=0.5, min_samples=5, metric='cosine').fit(feature_matrix)
    df['cluster'] = clustering.labels_
    
    print("Saving clustered results...")
    df.to_csv('clustered_products.csv', index=False)
    
    print(f"Clusters found: {len(set(df['cluster'])) - 1} (excluding noise)")
    print("Results saved to clustered_products.csv")
    
    return df

# Run the full pipeline
if __name__ == "__main__":
    df_standardized = clean_and_standardize_data()
    df_clustered = cluster_data()


Reading dataset...
Clustering manufacturer names...
Clustering product names...
Saving mappings to file...
Creating standardized dataset...

Original unique manufacturer names: 4414
Standardized unique manufacturer names: 3648
Original unique product names: 13426
Standardized unique product names: 8817
Loading standardized dataset...
Encoding text features with TF-IDF...
Scaling numerical features...
Combining features for clustering...
Applying DBSCAN...
Saving clustered results...
Clusters found: 8 (excluding noise)
Results saved to clustered_products.csv


In [4]:
df = pd.read_csv('products_standardized_fuzzy.csv')
df


,id,product_name,product_category_id,product_category_name,product_type_numeric,product_type_text,product_manufacturer_id,product_manufacturer_name,product_manufacturer_name_std,product_name_std
0,512842,Layers Mash,318,Feeds,1.0,kg,608.0,Lucky Feeds,Lucky Feeds,Layer mash
1,513291,Layers Mash,313,Minerals and Supplements,70.0,kg,2314.0,Mapato Feeds,Mapato Feeds,Layer mash
2,514788,Layers Mash,318,Feeds,10.0,kg,5311.0,murimi feeds,murimi feeds,Layer mash
3,514789,Layers Mash,318,Feeds,1.0,kg,5311.0,murimi feeds,murimi feeds,Layer mash
4,514845,Layers Mash,318,Feeds,5.0,kg,25878.0,Meru Central Ltd,Meru Central Ltd,Layer mash
...,...,...,...,...,...,...,...,...,...,...
49570,470588,bravo adult,318,Feeds,15.0,kg,23869.0,parikh packaging,parikh packaging,bravo adult
49571,483762,bravo adult,318,Feeds,15.0,kg,7285.0,bravo,bravo,bravo adult
49572,483820,bravo adult,318,Feeds,2.0,kg,7285.0,bravo,bravo,bravo adult
49573,484386,bravo adult,318,Feeds,8.0,kg,7285.0,bravo,bravo,bravo adult
